Testing pretrained Detectree2 model weights on OFO data
--------

In [1]:
from detectree2.models.outputs import project_to_geojson, stitch_crowns, clean_crowns
from detectree2.models.predict import predict_on_data
from detectron2.engine import DefaultPredictor
from detectree2.preprocessing.tiling import tile_data
from detectree2.models.train import setup_cfg

ERROR 1: PROJ: proj_create_from_database: Open of /ofo-share/repos-amritha/conda/envs/dtree2/share/proj failed


In [2]:
import rasterio
import geopandas as gpd

In [3]:
#trained_model = '/ofo-share/scratch-amritha/emerald_point_dtree2/dataset/train_outputs/model_6.pth' #finetuned
trained_model = '/ofo-share/repos-amritha/detectree2-code/230103_randresize_full.pth'  # path to saved weights
img_path = '/ofo-share/scratch-amritha/emerald_point_dtree2/dataset/rgb/ortho.tif'  # orthomosaic path
tiles_path = '/ofo-share/scratch-amritha/emerald_point_dtree2/finetuned/tiles/'  # folder to save the extracted tiles

In [4]:
data = rasterio.open(img_path)

In [5]:
buffer = 5  # buffer size around each tile
tile_width = 30
tile_height = 30
tiles_path = tiles_path +'tiles_'+ str(buffer) +'_'+ str(tile_width) +'_'+ str(tile_height) +'/'
tile_data(data, tiles_path, buffer, tile_width, tile_height, dtype_bool = True)

Tiling to 110 total tiles


/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/rasterio/mask.py:191: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/rasterio/mask.py:191: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/rasterio/mask.py:191: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/rasterio/mask.py:191: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  ou

Processed 50 tiles of 110 tiles


/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/rasterio/mask.py:191: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/rasterio/mask.py:191: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/rasterio/mask.py:191: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/rasterio/mask.py:191: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  ou

Processed 100 tiles of 110 tiles
Tiling complete


In [7]:
tiles_path = "/ofo-share/scratch-amritha/emerald_point_dtree2/tiles_5_30_30/"

In [8]:
cfg = setup_cfg(update_model = trained_model)
predict_on_data(tiles_path, predictor = DefaultPredictor(cfg))

/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, m

Predicting 100 files


/ofo-share/repos-amritha/conda/envs/dtree2/lib/python3.10/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Predicted 50 files of 100
Predicted 100 files of 100


In [9]:
project_to_geojson(tiles_path, tiles_path + "predictions/", tiles_path + "predictions_geo/")

Projecting 100 files
Projecting file 50 of 100: /ofo-share/scratch-amritha/emerald_point_dtree2/tiles_5_30_30/predictions/Prediction_ortho_752321_4317119_30_5_26910.json
Projecting file 100 of 100: /ofo-share/scratch-amritha/emerald_point_dtree2/tiles_5_30_30/predictions/Prediction_ortho_752471_4317059_30_5_26910.json


In [10]:
crowns = stitch_crowns(tiles_path + "predictions_geo/", 1) # stitch together predicted crowns from folder w geojson files
clean = clean_crowns(crowns, 0.6, confidence = 0) # set a confidence > 0 to filter out less confident crowns, IoU thresh 0.7

Stitching file 50 of 100: /ofo-share/scratch-amritha/emerald_point_dtree2/tiles_5_30_30/predictions_geo/Prediction_ortho_752471_4317029_30_5_26910.geojson
Stitching file 100 of 100: /ofo-share/scratch-amritha/emerald_point_dtree2/tiles_5_30_30/predictions_geo/Prediction_ortho_752261_4316999_30_5_26910.geojson
Cleaning 5488 crowns
0 / 5488 crowns cleaned
1000 / 5488 crowns cleaned
2000 / 5488 crowns cleaned
3000 / 5488 crowns cleaned
4000 / 5488 crowns cleaned
5000 / 5488 crowns cleaned


In [11]:
clean = clean[clean["Confidence_score"] > 0.5]  

In [12]:
clean = clean.set_geometry(clean.simplify(0.6))

In [13]:
clean.to_file(tiles_path + "/crowns_5_30_iou06_conf05.gpkg")

Loading tiles from path
--

In [4]:
crowns = stitch_crowns('/ofo-share/scratch-amritha/emerald_point_dtree2/tiles_5_20_20/' + "predictions_geo/", 1) # stitch together predicted crowns from folder w geojson files
clean = clean_crowns(crowns, 0.6, confidence = 0) # set a confidence > 0 to filter out less confident crowns, IoU thresh 0.7

Stitching file 50 of 240: /ofo-share/scratch-amritha/emerald_point_dtree2/tiles_5_20_20/predictions_geo/Prediction_ortho_752361_4317179_20_5_26910.geojson
Stitching file 100 of 240: /ofo-share/scratch-amritha/emerald_point_dtree2/tiles_5_20_20/predictions_geo/Prediction_ortho_752261_4317079_20_5_26910.geojson
Stitching file 150 of 240: /ofo-share/scratch-amritha/emerald_point_dtree2/tiles_5_20_20/predictions_geo/Prediction_ortho_752301_4316979_20_5_26910.geojson
Stitching file 200 of 240: /ofo-share/scratch-amritha/emerald_point_dtree2/tiles_5_20_20/predictions_geo/Prediction_ortho_752321_4317059_20_5_26910.geojson
Cleaning 11590 crowns
0 / 11590 crowns cleaned
1000 / 11590 crowns cleaned
2000 / 11590 crowns cleaned
3000 / 11590 crowns cleaned
4000 / 11590 crowns cleaned
5000 / 11590 crowns cleaned
6000 / 11590 crowns cleaned
7000 / 11590 crowns cleaned
8000 / 11590 crowns cleaned
9000 / 11590 crowns cleaned
10000 / 11590 crowns cleaned
11000 / 11590 crowns cleaned


In [5]:
clean = clean[clean["Confidence_score"] > 0.5]  
clean = clean.set_geometry(clean.simplify(0.6))
clean.to_file('/ofo-share/scratch-amritha/emerald_point_dtree2/tiles_5_20_20' + "/crowns_5_20_iou06_conf05.gpkg")